In [2]:
import os
if os.getcwd().endswith("notebooks"):
    os.chdir('..')

print("Current working directory: ", os.getcwd())
if not os.getcwd().endswith("California-Housing-ML"):
    raise ValueError("Please change working directory to 'path/toCalifornia-Housing-ML' before proceeding")
!pip install -r requirements.txt

Current working directory:  /Users/irellzane/MLprojects/California-Housing-ML


In [3]:
import pandas as pd
import numpy as np
from modules.preprocessing import preprocessing

NameError: name 'cat_pipeline' is not defined

In [42]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
clean_param_search_res(cv_res).head()

,n_clusters,max_features,split0,split1,split2,mean_test_rmse
1,45,9,41204,42108,42998,42103
8,32,7,41991,42557,43431,42660
5,42,4,42031,42968,43617,42872
0,41,16,42580,42869,43604,43018
2,23,8,42421,42841,44232,43165


```{toggle} Click the button to reveal!
:show:
Some hidden toggle content!

![](../images/cool.jpg)
```

## Train and Evaluate

## Save model

In [21]:
import joblib

joblib.dump(final_model, "models/my_california_housing_model.pkl.gz", compress='gzip')

['models/my_california_housing_model.pkl.gz']